In [13]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from IPython.display import display
%matplotlib inline

In [14]:
# Read data and drop redundant column.
data = pd.read_csv('epl_train.csv')

# Preview data.
data = data[data.MW > 3]

data.drop(['Unnamed: 0','HomeTeam', 'AwayTeam', 'Date', 'MW', 'HTFormPtsStr', 'ATFormPtsStr', 'FTHG', 'FTAG',
           'HTGS', 'ATGS', 'HTGC', 'ATGC','HomeTeamLP', 'AwayTeamLP','DiffPts','HTFormPts','ATFormPts',
           'HM4','HM5','AM4','AM5','HTLossStreak5','ATLossStreak5','HTWinStreak5','ATWinStreak5',
           'HTWinStreak3','HTLossStreak3','ATWinStreak3','ATLossStreak3'],1, inplace=True)

cleanup_nums = {"FTR":     {"H": 0, "D": 1, "A": 2}}
data = data.replace(cleanup_nums)
display(data.head())

,FTR,HTP,ATP,HM1,HM2,HM3,AM1,AM2,AM3,B365H,B365D,B365A,HTGD,ATGD,DiffFormPts,DiffLP
30,0,1.00,1.50,D,L,W,W,L,W,2.38,3.25,3.20,-0.50,0.00,-0.50,-2.0
31,0,2.25,1.00,W,W,W,D,W,L,1.36,4.75,9.00,1.50,0.00,1.25,-5.0
32,1,0.75,0.75,L,W,L,W,L,L,2.10,3.30,3.60,-0.50,-1.00,0.00,-4.0
33,0,1.50,0.75,W,W,L,L,L,W,1.73,3.60,5.25,0.50,-0.75,0.75,-6.0
34,0,0.75,0.75,W,L,L,L,L,W,1.73,3.60,5.25,-1.25,-1.00,0.00,-8.0


In [15]:
X_all = data.drop(['FTR'],1)
y_all = data['FTR']

cols = [['HTGD','ATGD','HTP','ATP','DiffLP']]

In [16]:
#last 3 wins
X_all.HM1 = X_all.HM1.astype('str')
X_all.HM2 = X_all.HM2.astype('str')
X_all.HM3 = X_all.HM3.astype('str')
X_all.AM1 = X_all.AM1.astype('str')
X_all.AM2 = X_all.AM2.astype('str')
X_all.AM3 = X_all.AM3.astype('str')

def features(X):
    
    temp = pd.DataFrame(index = X.index)

    # Check each column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised version
        temp = temp.join(col_data)
    
    return temp

X_all = features(X_all)
print("Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns)))

Processed feature columns (27 total features):
['HTP', 'ATP', 'HM1_D', 'HM1_L', 'HM1_W', 'HM2_D', 'HM2_L', 'HM2_W', 'HM3_D', 'HM3_L', 'HM3_W', 'AM1_D', 'AM1_L', 'AM1_W', 'AM2_D', 'AM2_L', 'AM2_W', 'AM3_D', 'AM3_L', 'AM3_W', 'B365H', 'B365D', 'B365A', 'HTGD', 'ATGD', 'DiffFormPts', 'DiffLP']


In [17]:
print("Feature values:")
display(X_all.head())

Feature values:


,HTP,ATP,HM1_D,HM1_L,HM1_W,HM2_D,HM2_L,HM2_W,HM3_D,HM3_L,...,AM3_D,AM3_L,AM3_W,B365H,B365D,B365A,HTGD,ATGD,DiffFormPts,DiffLP
30,1.00,1.50,1,0,0,0,1,0,0,0,...,0,0,1,2.38,3.25,3.20,-0.50,0.00,-0.50,-2.0
31,2.25,1.00,0,0,1,0,0,1,0,0,...,0,1,0,1.36,4.75,9.00,1.50,0.00,1.25,-5.0
32,0.75,0.75,0,1,0,0,0,1,0,1,...,0,1,0,2.10,3.30,3.60,-0.50,-1.00,0.00,-4.0
33,1.50,0.75,0,0,1,0,0,1,0,1,...,0,0,1,1.73,3.60,5.25,0.50,-0.75,0.75,-6.0
34,0.75,0.75,0,0,1,0,1,0,0,1,...,0,0,1,1.73,3.60,5.25,-1.25,-1.00,0.00,-8.0


In [48]:
from sklearn.model_selection import train_test_split

# Random split dataset into training and testing set 4:1 ratio.
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, 
                                                    test_size = 0.2,
                                                    random_state = 2,
                                                    stratify = y_all)

In [41]:
from time import time 
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print("Trained model in {:.4f} seconds".format(end - start))

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    
    end = time()
    # Print and return results
    print("Made predictions in {:.4f} seconds.".format(end - start))
    
    return f1_score(target, y_pred, pos_label='H',average='micro'), sum(target == y_pred) / float(len(y_pred))


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print("Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train)))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    f1, acc = predict_labels(clf, X_train, y_train)
    print(f1, acc)
    print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
    
    f1, acc = predict_labels(clf, X_test, y_test)
    print("F1 score and accuracy score for test set: {:.4f} , {:.4f}.".format(f1 , acc))

In [50]:
# Initialize SVM
clf_svm = SVC(kernel='rbf')
train_predict(clf_svm, X_train, y_train, X_test, y_test)

Training a SVC using a training set size of 3079. . .
Trained model in 0.6154 seconds
Made predictions in 0.9315 seconds.
0.5576485872036375 0.5576485872036375
F1 score and accuracy score for training set: 0.5576 , 0.5576.
Made predictions in 0.2045 seconds.
F1 score and accuracy score for test set: 0.5701 , 0.5701.


In [51]:
# Hyper parameter tuning
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 200],
              'gamma': [1, 0.1, 0.01, 0.001,],
              'kernel': ['rbf'],
              'degree':[1,2,3,4]
              }
              
# fitting the model for grid search
grid = GridSearchCV(SVC(), param_grid, refit = True, verbose = 3)
grid.fit(X_train, y_train)

# print best parameter after tuning
print("Best Parameters: ",grid.best_params_)
print("Best Estimators: ",grid.best_estimator_)
print(classification_report(y_test, grid.predict(X_test)))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV 1/5] END C=0.1, degree=1, gamma=1, kernel=rbf;, score=0.468 total time=   0.6s
[CV 2/5] END C=0.1, degree=1, gamma=1, kernel=rbf;, score=0.468 total time=   0.6s
[CV 3/5] END C=0.1, degree=1, gamma=1, kernel=rbf;, score=0.468 total time=   0.7s
[CV 4/5] END C=0.1, degree=1, gamma=1, kernel=rbf;, score=0.466 total time=   0.7s
[CV 5/5] END C=0.1, degree=1, gamma=1, kernel=rbf;, score=0.467 total time=   0.8s
[CV 1/5] END C=0.1, degree=1, gamma=0.1, kernel=rbf;, score=0.542 total time=   0.5s
[CV 2/5] END C=0.1, degree=1, gamma=0.1, kernel=rbf;, score=0.567 total time=   0.5s
[CV 3/5] END C=0.1, degree=1, gamma=0.1, kernel=rbf;, score=0.542 total time=   0.5s
[CV 4/5] END C=0.1, degree=1, gamma=0.1, kernel=rbf;, score=0.531 total time=   0.5s
[CV 5/5] END C=0.1, degree=1, gamma=0.1, kernel=rbf;, score=0.550 total time=   0.5s
[CV 1/5] END C=0.1, degree=1, gamma=0.01, kernel=rbf;, score=0.552 total time=   0.4s
[CV 2/5] END

[CV 2/5] END C=1, degree=1, gamma=0.001, kernel=rbf;, score=0.576 total time=   0.5s
[CV 3/5] END C=1, degree=1, gamma=0.001, kernel=rbf;, score=0.555 total time=   0.5s
[CV 4/5] END C=1, degree=1, gamma=0.001, kernel=rbf;, score=0.545 total time=   0.5s
[CV 5/5] END C=1, degree=1, gamma=0.001, kernel=rbf;, score=0.559 total time=   0.4s
[CV 1/5] END C=1, degree=2, gamma=1, kernel=rbf;, score=0.466 total time=   0.7s
[CV 2/5] END C=1, degree=2, gamma=1, kernel=rbf;, score=0.468 total time=   0.8s
[CV 3/5] END C=1, degree=2, gamma=1, kernel=rbf;, score=0.463 total time=   0.8s
[CV 4/5] END C=1, degree=2, gamma=1, kernel=rbf;, score=0.459 total time=   0.7s
[CV 5/5] END C=1, degree=2, gamma=1, kernel=rbf;, score=0.455 total time=   0.7s
[CV 1/5] END C=1, degree=2, gamma=0.1, kernel=rbf;, score=0.534 total time=   0.6s
[CV 2/5] END C=1, degree=2, gamma=0.1, kernel=rbf;, score=0.563 total time=   0.6s
[CV 3/5] END C=1, degree=2, gamma=0.1, kernel=rbf;, score=0.542 total time=   0.5s
[CV 4/

[CV 1/5] END C=10, degree=2, gamma=0.001, kernel=rbf;, score=0.554 total time=   0.4s
[CV 2/5] END C=10, degree=2, gamma=0.001, kernel=rbf;, score=0.578 total time=   0.5s
[CV 3/5] END C=10, degree=2, gamma=0.001, kernel=rbf;, score=0.562 total time=   0.5s
[CV 4/5] END C=10, degree=2, gamma=0.001, kernel=rbf;, score=0.552 total time=   0.4s
[CV 5/5] END C=10, degree=2, gamma=0.001, kernel=rbf;, score=0.558 total time=   0.6s
[CV 1/5] END C=10, degree=3, gamma=1, kernel=rbf;, score=0.463 total time=   0.7s
[CV 2/5] END C=10, degree=3, gamma=1, kernel=rbf;, score=0.469 total time=   0.8s
[CV 3/5] END C=10, degree=3, gamma=1, kernel=rbf;, score=0.469 total time=   1.0s
[CV 4/5] END C=10, degree=3, gamma=1, kernel=rbf;, score=0.459 total time=   0.8s
[CV 5/5] END C=10, degree=3, gamma=1, kernel=rbf;, score=0.467 total time=   1.1s
[CV 1/5] END C=10, degree=3, gamma=0.1, kernel=rbf;, score=0.477 total time=   0.8s
[CV 2/5] END C=10, degree=3, gamma=0.1, kernel=rbf;, score=0.505 total time=

[CV 3/5] END C=100, degree=3, gamma=0.01, kernel=rbf;, score=0.519 total time=   0.8s
[CV 4/5] END C=100, degree=3, gamma=0.01, kernel=rbf;, score=0.528 total time=   0.8s
[CV 5/5] END C=100, degree=3, gamma=0.01, kernel=rbf;, score=0.564 total time=   0.8s
[CV 1/5] END C=100, degree=3, gamma=0.001, kernel=rbf;, score=0.545 total time=   0.5s
[CV 2/5] END C=100, degree=3, gamma=0.001, kernel=rbf;, score=0.578 total time=   0.5s
[CV 3/5] END C=100, degree=3, gamma=0.001, kernel=rbf;, score=0.557 total time=   0.5s
[CV 4/5] END C=100, degree=3, gamma=0.001, kernel=rbf;, score=0.545 total time=   0.5s
[CV 5/5] END C=100, degree=3, gamma=0.001, kernel=rbf;, score=0.563 total time=   0.5s
[CV 1/5] END C=100, degree=4, gamma=1, kernel=rbf;, score=0.463 total time=   0.6s
[CV 2/5] END C=100, degree=4, gamma=1, kernel=rbf;, score=0.469 total time=   0.6s
[CV 3/5] END C=100, degree=4, gamma=1, kernel=rbf;, score=0.469 total time=   0.6s
[CV 4/5] END C=100, degree=4, gamma=1, kernel=rbf;, score=

[CV 5/5] END C=200, degree=4, gamma=0.1, kernel=rbf;, score=0.499 total time=   0.8s
[CV 1/5] END C=200, degree=4, gamma=0.01, kernel=rbf;, score=0.502 total time=   1.1s
[CV 2/5] END C=200, degree=4, gamma=0.01, kernel=rbf;, score=0.541 total time=   1.2s
[CV 3/5] END C=200, degree=4, gamma=0.01, kernel=rbf;, score=0.518 total time=   1.1s
[CV 4/5] END C=200, degree=4, gamma=0.01, kernel=rbf;, score=0.515 total time=   1.1s
[CV 5/5] END C=200, degree=4, gamma=0.01, kernel=rbf;, score=0.556 total time=   1.2s
[CV 1/5] END C=200, degree=4, gamma=0.001, kernel=rbf;, score=0.542 total time=   0.6s
[CV 2/5] END C=200, degree=4, gamma=0.001, kernel=rbf;, score=0.581 total time=   0.7s
[CV 3/5] END C=200, degree=4, gamma=0.001, kernel=rbf;, score=0.565 total time=   0.6s
[CV 4/5] END C=200, degree=4, gamma=0.001, kernel=rbf;, score=0.549 total time=   0.6s
[CV 5/5] END C=200, degree=4, gamma=0.001, kernel=rbf;, score=0.556 total time=   0.7s
Best Parameters:  {'C': 10, 'degree': 1, 'gamma': 

In [52]:
# Metric results after tuning
clf_svm = SVC(gamma=0.01,C=10,degree=1, kernel='rbf')
train_predict(clf_svm, X_train, y_train, X_test, y_test)

Training a SVC using a training set size of 3079. . .
Trained model in 0.6862 seconds
Made predictions in 0.9914 seconds.
0.582656706722962 0.582656706722962
F1 score and accuracy score for training set: 0.5827 , 0.5827.
Made predictions in 0.1995 seconds.
F1 score and accuracy score for test set: 0.5779 , 0.5779.


In [53]:
#Importing test set
from sklearn.metrics import classification_report, confusion_matrix
epl = pd.read_csv('epl_test.csv')

epl = epl[epl.MW > 3]

epl.drop(['Unnamed: 0','HomeTeam', 'AwayTeam', 'Date', 'MW', 'HTFormPtsStr', 'ATFormPtsStr', 'FTHG', 'FTAG',
           'HTGS', 'ATGS', 'HTGC', 'ATGC','HomeTeamLP', 'AwayTeamLP','DiffPts','HTFormPts','ATFormPts',
           'HM4','HM5','AM4','AM5','HTLossStreak5','ATLossStreak5','HTWinStreak5','ATWinStreak5',
           'HTWinStreak3','HTLossStreak3','ATWinStreak3','ATLossStreak3'],1, inplace=True)
cleanup_nums = {"FTR":     {"H": 0, "D": 1, "A": 2}}
epl = epl.replace(cleanup_nums)
X1_all = epl.drop(['FTR'],1)
y1_all = epl['FTR']
X1_all.HM1 = X1_all.HM1.astype('str')
X1_all.HM2 = X1_all.HM2.astype('str')
X1_all.HM3 = X1_all.HM3.astype('str')
X1_all.AM1 = X1_all.AM1.astype('str')
X1_all.AM2 = X1_all.AM2.astype('str')
X1_all.AM3 = X1_all.AM3.astype('str')

#we want continous vars that are integers for our input data, so lets remove any categorical vars
def preprocess_features(X):
    ''' Preprocesses the football data and converts catagorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix = col)
                    
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X1_all = preprocess_features(X1_all)
print("Processed feature columns ({} total features):\n{}".format(len(X1_all.columns), list(X1_all.columns)))

Processed feature columns (27 total features):
['HTP', 'ATP', 'HM1_D', 'HM1_L', 'HM1_W', 'HM2_D', 'HM2_L', 'HM2_W', 'HM3_D', 'HM3_L', 'HM3_W', 'AM1_D', 'AM1_L', 'AM1_W', 'AM2_D', 'AM2_L', 'AM2_W', 'AM3_D', 'AM3_L', 'AM3_W', 'B365H', 'B365D', 'B365A', 'HTGD', 'ATGD', 'DiffFormPts', 'DiffLP']


In [54]:
# Checking metrics using test set
clf_svm = SVC(gamma=0.01,C=10,degree=1, kernel='rbf')
clf_svm.fit(X1_all,y1_all)
f1, acc = predict_labels(clf_svm, X1_all, y1_all)
print("F1 score and accuracy score for training set: {:.4f} , {:.4f}.".format(f1 , acc))
print(classification_report(y1_all, clf_svm.predict(X1_all)))

Made predictions in 0.0170 seconds.
F1 score and accuracy score for training set: 0.6600 , 0.6600.
              precision    recall  f1-score   support

           0       0.66      0.73      0.70       133
           1       0.82      0.33      0.47        81
           2       0.63      0.79      0.70       136

    accuracy                           0.66       350
   macro avg       0.70      0.62      0.62       350
weighted avg       0.68      0.66      0.64       350



In [56]:
#Predictions
y1_predict = clf_svm.predict(X1_all)
print(y1_predict)

[2 0 2 0 0 0 2 2 2 0 0 2 0 0 0 2 0 0 2 1 0 1 0 2 2 2 1 2 2 2 2 0 2 0 0 0 1
 2 1 0 2 2 0 0 2 0 1 2 2 0 2 0 2 2 0 2 0 2 0 2 0 2 0 2 2 1 0 0 2 0 2 0 2 2
 0 0 0 2 2 0 1 2 0 2 2 0 0 2 1 1 0 2 0 2 1 0 2 2 2 0 1 0 2 0 2 0 0 2 2 1 2
 0 2 0 0 0 2 2 1 2 0 2 2 0 1 2 0 2 0 1 2 2 2 2 2 2 2 0 0 0 0 2 0 2 0 2 2 0
 0 0 2 0 2 0 2 2 2 2 2 2 0 0 2 0 2 0 0 0 1 2 2 0 0 2 2 0 2 0 0 2 2 2 2 0 2
 2 1 0 2 1 0 2 2 2 0 2 2 0 2 2 2 0 0 0 0 0 2 2 2 0 2 0 0 2 2 0 0 2 0 0 0 1
 1 0 0 2 2 0 0 2 2 2 2 2 2 2 2 1 2 1 2 2 0 0 0 0 1 1 1 2 2 2 0 0 0 2 2 2 2
 2 2 0 2 2 0 1 0 0 1 2 2 0 0 2 2 0 2 2 0 2 0 2 0 0 0 2 0 0 2 0 2 0 2 0 0 2
 1 0 2 2 2 0 2 0 0 2 2 0 1 2 2 0 1 2 2 0 0 0 0 2 1 2 2 2 0 2 2 0 2 0 0 2 2
 2 0 0 0 1 2 2 0 2 2 0 2 0 0 2 0 2]


In [57]:
# Checking returns using Logistic Regression
funds = 100
wager = 10
favourites = 0
no_bets = 0
min_diff = 0.00

for i in range(len(X1_all)):
    addition = 30+i
    prediction = y1_predict[i]
    print('\nMatch',i+1)
    print('\nPrediction', prediction)
    print('Actual', y1_all.iloc[i])
    print('Favourite', np.argmin([epl.loc[addition,'B365H'], epl.loc[addition,'B365D'], epl.loc[addition,'B365A']]))
    print('Home, Draw and Away odds', epl.loc[addition,'B365H'], epl.loc[addition,'B365D'], epl.loc[addition,'B365A'])      
          
    
    if prediction == 2:
            if prediction == np.argmin([epl.loc[addition,'B365H'], epl.loc[addition,'B365D'], epl.loc[addition,'B365A']]): 
                                
                favourites +=1
                
            if  prediction == y1_all.iloc[i]:
                funds += (wager * epl.loc[addition,'B365A']) - wager
            else:
                funds -= wager
            
    elif prediction == 1:
            if prediction == np.argmin([epl.loc[addition,'B365H'], epl.loc[addition,'B365D'], epl.loc[addition,'B365A']]):  

                favourites +=1
            if  prediction == y1_all.iloc[i]:
                funds +=( wager * epl.loc[addition,'B365D']) - wager
            else:
                funds -= wager

    else:

            if prediction == np.argmin([epl.loc[addition,'B365H'], epl.loc[addition,'B365D'], epl.loc[addition,'B365A']]):
                favourites +=1
            if  prediction == y1_all.iloc[i]:
                funds += (wager * epl.loc[addition,'B365A']) - wager
            else:
                funds -= wager
   
    print('Funds', funds)
    

print(f'No bet placed {no_bets} times')


Match 1

Prediction 2
Actual 1
Favourite 2
Home, Draw and Away odds 7.5 5.25 1.36
Funds 90

Match 2

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 2.4 3.2 3.1
Funds 111.0

Match 3

Prediction 2
Actual 2
Favourite 0
Home, Draw and Away odds 1.61 4.2 5.25
Funds 153.5

Match 4

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.72 3.8 4.75
Funds 191.0

Match 5

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.57 4.0 6.0
Funds 241.0

Match 6

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.5 4.0 7.5
Funds 306.0

Match 7

Prediction 2
Actual 2
Favourite 0
Home, Draw and Away odds 1.85 3.8 4.0
Funds 336.0

Match 8

Prediction 2
Actual 0
Favourite 2
Home, Draw and Away odds 10.0 5.5 1.3
Funds 326.0

Match 9

Prediction 2
Actual 1
Favourite 2
Home, Draw and Away odds 3.75 4.2 1.83
Funds 316.0

Match 10

Prediction 0
Actual 1
Favourite 0
Home, Draw and Away odds 1.45 4.75 6.5
Funds 306.0

Match 11

Prediction 0
Actual 0
Favourite 0
Home, Draw and

Home, Draw and Away odds 1.61 4.2 5.0
Funds 2233.7000000000007

Match 120

Prediction 2
Actual 2
Favourite 2
Home, Draw and Away odds 3.25 3.5 2.2
Funds 2245.7000000000007

Match 121

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 2.37 3.1 3.25
Funds 2268.2000000000007

Match 122

Prediction 2
Actual 1
Favourite 0
Home, Draw and Away odds 2.1 3.6 3.4
Funds 2258.2000000000007

Match 123

Prediction 2
Actual 2
Favourite 2
Home, Draw and Away odds 3.6 3.6 2.0
Funds 2268.2000000000007

Match 124

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.53 4.2 6.25
Funds 2320.7000000000007

Match 125

Prediction 1
Actual 1
Favourite 2
Home, Draw and Away odds 12.0 6.0 1.25
Funds 2370.7000000000007

Match 126

Prediction 2
Actual 2
Favourite 0
Home, Draw and Away odds 2.1 3.5 3.5
Funds 2395.7000000000007

Match 127

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.72 4.2 4.2
Funds 2427.7000000000007

Match 128

Prediction 2
Actual 0
Favourite 0
Home, Draw and Away

Favourite 2
Home, Draw and Away odds 2.9 3.3 2.45
Funds 5101.500000000002

Match 261

Prediction 2
Actual 2
Favourite 0
Home, Draw and Away odds 1.22 6.0 15.0
Funds 5241.500000000002

Match 262

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.5 4.33 6.5
Funds 5296.500000000002

Match 263

Prediction 2
Actual 2
Favourite 2
Home, Draw and Away odds 6.0 4.2 1.53
Funds 5301.800000000002

Match 264

Prediction 2
Actual 2
Favourite 2
Home, Draw and Away odds 3.1 3.7 2.15
Funds 5313.300000000002

Match 265

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 2.05 3.3 3.8
Funds 5341.300000000002

Match 266

Prediction 1
Actual 1
Favourite 2
Home, Draw and Away odds 5.5 4.0 1.6
Funds 5371.300000000002

Match 267

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 2.25 3.3 3.3
Funds 5394.300000000002

Match 268

Prediction 0
Actual 0
Favourite 0
Home, Draw and Away odds 1.61 4.0 5.5
Funds 5439.300000000002

Match 269

Prediction 1
Actual 1
Favourite 0
Home, Draw and A